In [1]:
import pandas as pd
import datetime
import numpy as np
import pygsheets
import polyline
#from geojson import Polygon
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from folium.plugins import MarkerCluster
import geopandas as gpd
from sklearn import preprocessing
import shapely
import pickle
import folium
import string

#local import#
from bestPlace.bestPlace import location
import bestPlace.helper as helper


In [14]:
import importlib
importlib.reload(helper)
import bestPlace.helper as helper

In [3]:
house_addresses = './tests/test_data_houses.csv'
places_of_importance = './data_places_of_importance.csv'
commuting_locations = './tests/test_data_commuting_stations.csv'
location_addresses = helper.read_address_csv(house_addresses)
house_locations = []

In [4]:
gc = pygsheets.authorize(client_secret='./client_secret_80237418460-ojb5d5fh62tiup3286i6vaejo715m6l9.apps.googleusercontent.com.json')
sh = gc.open('zillow_properties')
wks = sh[0]
addresses = wks.get_all_records()
location_addresses = pd.DataFrame(addresses)
#filtered_addresses = location_addresses 

In [5]:
current_address = []
for house in house_locations:
    current_address.append(house.address)
current_address
location_addresses[~location_addresses['address'].isin(current_address)]
filtered_addresses = location_addresses[~location_addresses['address'].isin(current_address)]

In [6]:
for index, row in filtered_addresses.iterrows():
    house_data = {'address':row['address'],'state':row['state'],'locationType':row['locationType'],'zip':row['zip']}
    print(house_data)
    house_locations.append(
                            location(house_data,places_of_importance,commuting_locations)
    )
    

{'address': '7 Orchard Ln, Katonah', 'state': 'NY', 'locationType': 'house', 'zip': 10536}
{'address': '6 Applegate Way, Ossining', 'state': 'NY', 'locationType': 'house', 'zip': 10562}
{'address': '82 Todd Rd, Katonah', 'state': 'NY', 'locationType': 'house', 'zip': 10536}
{'address': '57 Highland Ave, North Tarrytown', 'state': 'NY', 'locationType': 'house', 'zip': 10591}
{'address': '90 Carriage Dr, Red Hook', 'state': 'NY', 'locationType': 'house', 'zip': 12571}
{'address': '47 Parry Rd, Stamford', 'state': 'CT', 'locationType': 'house', 'zip': 6907}
{'address': '43 High St, Croton On Hudson', 'state': 'NY', 'locationType': 'house', 'zip': 10520}
{'address': '58 Central Ave, North Tarrytown', 'state': 'NY', 'locationType': 'house', 'zip': 10591}
{'address': '14 Clinton St, Sleepy Hollow', 'state': 'NY', 'locationType': 'house', 'zip': 10591}
{'address': '167 Benedict Ave, North Tarrytown', 'state': 'NY', 'locationType': 'house', 'zip': 10591}
{'address': '2736 independence avenue a

In [17]:
NYC_COORDINATES = (41.03, -73.76)

# for speed purposes
MAX_RECORDS = 1000

# create empty map zoomed in on NYC
map = folium.Map(location=NYC_COORDINATES, zoom_start=10)
mc = MarkerCluster()
geo_df = gpd.GeoDataFrame()
# add a marker for every record in the filtered data, use a clustered view
for house in house_locations:
    try:
        agg_df = helper.product_output_df(house)
        geo_df = helper.append_geo_polygons(house,geo_df,agg_df)
        html = pd.DataFrame([{'address':house.address}]).to_html() + "\n\n" \
                + house.df_travel_aggregate.to_html() + "\n\n" \
                + house.df_places_of_importance[['destination','duration']].to_html()
        iframe = folium.IFrame(html=html, width=500, height=300)
        popup = folium.Popup(iframe, max_width=2650)
        mc.add_child(folium.Marker(location=[house.address_gps['lat'],house.address_gps['lng']],popup=popup))
        
    except:
        print('failed ',house.address)
map.add_child(mc)  


failed  2736 independence avenue apt4a, bronx
failed  3720 Independence Ave APT 1D, Bronx
failed  1976 Lafontaine Ave APT 1C
failed  3 Fordham Hill Oval APT 7F,bronx
failed  439 E 143rd St, Bronx


In [24]:
def heatmap_data(house_locations,field):
    data = []
    for house in house_locations:
        try:
            aaa = helper.product_output_df(house)
            value = aaa[aaa['field']==field]['values'].values[0]
            ggg = [house.address,house.address_gps['lat'],house.address_gps['lng'],float(value)]
            data.append(ggg)
        except:
            print('failed' + house.address)
    df = pd.DataFrame(data)
    df.columns = ['address','lat','lng','duration']
    return(df)
        
                                                                           
        
heatmap_data = heatmap_data(house_locations,'89 E 42nd St, New York, NY 10017')



failed2736 independence avenue apt4a, bronx
failed3720 Independence Ave APT 1D, Bronx
failed1976 Lafontaine Ave APT 1C
failed3 Fordham Hill Oval APT 7F,bronx
failed439 E 143rd St, Bronx


In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
heatmap_data['duration_scaled'] = scaler.fit_transform(heatmap_data['duration'].values.reshape(-1,1))

TypeError: 'function' object is not subscriptable

In [20]:
map_2 = folium.Map(location=NYC_COORDINATES, zoom_start=10)
heatmap = folium.plugins.HeatMap(heatmap_data[['lat','lng','duration_scaled']].values.tolist(),
                                 radius = 10,
                                 max_val=1,
                                 max_zoom=10,
                                 gradient={.4: 'blue', .65: 'lime', 1: 'red'})
map_2.add_child(heatmap)


TypeError: 'function' object is not subscriptable

In [21]:
vals = geo_df[['districtID','districtName','school_ranking']]
vals.drop_duplicates()

,districtID,districtName,school_ranking
0,3616080,Katonah-Lewisboro Union Free School District,95.0
0,3622020,Ossining Union Free School District,19.0
0,3628650,Union Free School District Of The Tarrytowns,38.0
0,3624240,Red Hook Central School District,83.0
0,0904320,Stamford School District,21.0
0,3608580,Croton-Harmon Union Free School District,90.0
0,3606990,Chappaqua Central School District,98.0
0,3616620,Lakeland Central School District,81.0
0,3621180,North Salem Central School District,92.0
0,0902700,New Canaan School District,99.0


In [22]:
geo_df["geometry"] = geo_df["geometry"].apply(lambda geom: geom.wkb)
geo_df = geo_df.drop_duplicates()
# convert back to shapely geometry
geo_df["geometry"] = geo_df["geometry"].apply(lambda geom: shapely.wkb.loads(geom))

In [23]:
# Add the color for the chloropleth:
folium.Choropleth(
 geo_data=geo_df.to_json(),
 name='choropleth',
 data=vals.drop_duplicates(),
 columns=['districtID','school_ranking'],
 key_on='feature.properties.districtID',
 fill_color='YlGn',
 threshold_scale=[0, 25, 50, 60, 70, 80, 85, 90, 95, 100],
 fill_opacity=0.7,
 line_opacity=0.9,
 legend_name='School (%)'
).add_to(map)
map

In [202]:
bins = list(vals['school_ranking'].quantile([0, 0.25, 0.5, 0.75, 1]))

,address
0,"4 Lake Gilead Rd, Carmel"
